<a href="https://colab.research.google.com/github/Shristythapa/Text2Playlist/blob/main/Copy_of_t5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

! pip install -U accelerate
! pip install -U transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset

# Step 1: Load and Prepare the Data
df = pd.read_csv('/content/drive/MyDrive/final_playlist.csv')




In [ ]:
df.head()

,playlistName,trackName,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Soundtracks,L'Estasi Dell'oro (The Ecstasy Of Gold) - 2004...,spotify:track:6PrKZUXJPmBiobMN44yR8Y,0.134,0.474,9.0,-13.864,0.0,0.0518,0.826,0.580000,0.1950,0.133,98.854
1,Soundtracks,Woo Hoo,spotify:track:6GA8fbPKP6KZ3fP75AIjRI,0.557,0.714,2.0,-12.769,1.0,0.0454,0.707,0.560000,0.6050,0.949,128.066
2,Soundtracks,Twisted Nerve,spotify:track:3Y9wdXMh6sNt9fUwUqTqb7,0.622,0.166,4.0,-14.063,0.0,0.0588,0.982,0.109000,0.1110,0.224,115.022
3,Soundtracks,Django (Vocal English Version),spotify:track:7e8njbgEj7safbzhmHwMG2,0.459,0.569,2.0,-9.968,0.0,0.0295,0.570,0.000000,0.0914,0.468,97.267
4,Soundtracks,Lo Chiamavano King (His Name Is King),spotify:track:342v3Is0g1CfidoxekA4f8,0.519,0.662,9.0,-4.398,0.0,0.0513,0.527,0.000003,0.1060,0.607,90.222


In [ ]:
# Columns to be normalized
features_to_standardize = [
    'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
    'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'
]

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Fit and transform the selected features
df[features_to_standardize] = scaler.fit_transform(df[features_to_standardize])


In [ ]:
from num2words import num2words
# Function to convert numerical values to words
def convert_to_words(value):
    try:
        # Convert to words if it's a number
        return num2words(value)
    except (TypeError, ValueError):
        # Return the value as is if it's not a number
        return value
# List of columns to convert
columns_to_convert = [
    'danceability', 'energy', 'key', 'loudness', 'mode',
    'speechiness', 'acousticness', 'instrumentalness',
    'liveness', 'valence', 'tempo'
]

# Apply the conversion only to specified columns using apply
df[columns_to_convert] = df[columns_to_convert].applymap(convert_to_words)

15560

In [ ]:
# # Save the normalized DataFrame to a new CSV file
df.to_csv('/content/drive/MyDrive/playlists_scaliled_worded.csv', index=False)




In [ ]:
df

In [ ]:
# Split the data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Step 2: Create Input-Output Pairs
features = ['danceability', 'energy', 'key', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']


In [ ]:

# Step 2: Create Input-Output Pairs for T5
def create_input_output_pairs(df):
    inputs = [
        row['playlistName'] for _, row in df.iterrows()
    ]
    outputs = [
        ' | '.join([f'{key}: {format_feature_value(row[key])}' for key in features])
        for _, row in df.iterrows()
    ]
    return inputs, outputs

train_inputs, train_outputs = create_input_output_pairs(train_df)
test_inputs, test_outputs = create_input_output_pairs(test_df)


In [ ]:
train_inputs[0]

'covers'

In [ ]:
train_outputs[0]

'danceability: zero point five zero seven seven zero six zero eight four nine three five one four six five energy: zero point one two zero five eight six three two zero nine zero five six four nine two eight key: one point zero six three one seven zero five three three five four five zero eight eight two mode: minus one point three eight nine zero two five four nine five nine zero seven four four six one speechiness: zero point zero seven one nine six four six six nine nine two one zero six three two acousticness: zero point six seven two four seven one one nine three seven five seven one three two nine instrumentalness: zero point two nine four eight five nine one two five one six eight three five eight seven six liveness: two point zero eight three three seven six eight five two four three eight eight three eight five valence: zero point four three eight six three zero seven three one six seven eight five eight seven three tempo: zero point six four two eight seven four four five zer

In [ ]:
# Step 2: Create Input-Output Pairs for T5
def create_input_output_pairs(df):
    features = [
        'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
        'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'
    ]

    def format_feature_value(value):
        if isinstance(value, float):
            return f"{value:.4f}"  # Limit floats to 4 decimal places
        return str(value)

    inputs = [row['playlistName'] for _, row in df.iterrows()]
    outputs = [
        ' | '.join([f'{key}: {format_feature_value(row[key])}' for key in features])
        for _, row in df.iterrows()
    ]

    return inputs, outputs

# Assuming train_df and test_df are your training and testing dataframes
train_inputs, train_outputs = create_input_output_pairs(train_df)
test_inputs, test_outputs = create_input_output_pairs(test_df)

# Example output
print("Example Input:", train_inputs[0])
print("Example Output:", train_outputs[0])

Example Input: covers
Example Output: danceability: zero point five zero seven seven zero six zero eight four nine three five one four six five | energy: zero point one two zero five eight six three two zero nine zero five six four nine two eight | key: one point zero six three one seven zero five three three five four five zero eight eight two | loudness: zero point one six one two zero one six four seven zero zero nine two nine two seven four | mode: minus one point three eight nine zero two five four nine five nine zero seven four four six one | speechiness: zero point zero seven one nine six four six six nine nine two one zero six three two | acousticness: zero point six seven two four seven one one nine three seven five seven one three two nine | instrumentalness: zero point two nine four eight five nine one two five one six eight three five eight seven six | liveness: two point zero eight three three seven six eight five two four three eight eight three eight five | valence: zero

In [ ]:

# Step 3: Tokenize Inputs and Outputs using T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base')



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
train_encodings = tokenizer(train_inputs, padding=True, truncation=True, return_tensors="pt")
train_labels = tokenizer(train_outputs, padding=True, truncation=True, return_tensors="pt")

test_encodings = tokenizer(test_inputs, padding=True, truncation=True, return_tensors="pt")
test_labels = tokenizer(test_outputs, padding=True, truncation=True, return_tensors="pt")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
type(train_encodings)

transformers.tokenization_utils_base.BatchEncoding

In [ ]:

# Define a custom Dataset class for PyTorch
class PlaylistDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels['input_ids'][idx]
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Create the datasets
train_dataset = PlaylistDataset(train_encodings, train_labels)
test_dataset = PlaylistDataset(test_encodings, test_labels)

In [ ]:
train_dataset

In [ ]:
# Step 5: Initialize and Train the Model
# Initialize the T5 model for sequence-to-sequence task
model = T5ForConditionalGeneration.from_pretrained('t5-base')


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:

# Define training arguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/music_playlist_rec_part2/results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/music_playlist_rec_part2/logs',
    eval_strategy="epoch"
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.687100,0.684220


Epoch,Training Loss,Validation Loss
1,0.687100,0.684220
2,0.674800,0.670435
3,0.668500,0.666513


TrainOutput(global_step=9336, training_loss=0.8036546698981304, metrics={'train_runtime': 2706.6553, 'train_samples_per_second': 13.797, 'train_steps_per_second': 3.449, 'total_flos': 577406259855360.0, 'train_loss': 0.8036546698981304, 'epoch': 3.0})

In [ ]:
# Evaluate the model on the test dataset
evaluation_results = trainer.evaluate()

# Print evaluation results
print("Evaluation results:")
for key, value in evaluation_results.items():
    print(f"{key}: {value}")

Evaluation results:
eval_loss: 2.3079587663232815e-07
eval_runtime: 38.5043
eval_samples_per_second: 80.822
eval_steps_per_second: 20.206
epoch: 3.0


In [ ]:
# Save evaluation results to a file
results_filename = '/content/drive/MyDrive/music_playlist_rec_part2/evaluation_results.txt'

with open(results_filename, 'w') as f:
    f.write("Evaluation results:\n")
    for key, value in evaluation_results.items():
        f.write(f"{key}: {value}\n")

print(f"Evaluation results saved to {results_filename}.")

Evaluation results saved to /content/drive/MyDrive/music_playlist_rec_part2/evaluation_results.txt.


In [ ]:
!pip uninstall transformers accelerate -y


Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
Found existing installation: accelerate 0.31.0
Uninstalling accelerate-0.31.0:
  Successfully uninstalled accelerate-0.31.0


In [ ]:

! pip install -U accelerate


In [ ]:
! pip install -U transformers

  Using cached transformers-4.41.2-py3-none-any.whl (9.1 MB)


In [ ]:
import transformers
import accelerate

print("Transformers version:", transformers.__version__)
print("Accelerate version:", accelerate.__version__)


Transformers version: 4.41.2
Accelerate version: 0.31.0


In [ ]:
!pip install transformers[torch] accelerate -U


In [ ]:
from transformers import T5ForConditionalGeneration

# Load the fine-tuned model
model_path = '/content/drive/MyDrive/music_playlist_rec_part2/results/checkpoint-3000'  # Replace with the actual path to your fine-tuned model checkpoint
model = T5ForConditionalGeneration.from_pretrained(model_path)


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the fine-tuned model
model_path = '/content/drive/MyDrive/music_playlist_rec_part2/results/checkpoint-3000'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Example input playlist description
playlist_description = "playlist: 2015 favorites"

# Construct the input prompt for the model
prompt = f"{playlist_description}"

# Tokenize the input prompt
inputs = tokenizer(prompt, return_tensors="pt")

# Generate music features based on the input prompt
outputs = model.generate(input_ids=inputs['input_ids'],
                         attention_mask=inputs['attention_mask'],
                         max_length=100,  # Adjust max_length based on expected output length
                         num_beams=5,    # Adjust num_beams for beam search (optional)
                         early_stopping=True  # Enable early stopping to return when the model's output ends
                        )

# Decode the generated output tokens
generated_music_features = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Generated music features:", generated_music_features)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generated music features: 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015 favorites 2015
